# Comparing Expert, MAB, FB and FR algorithms performances

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../')

In [3]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout

In [4]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [6]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)

<module 'source.players.belief_max' from '..\\source\\players\\belief_max.py'>

### Creating configurations to save into a csv file

In [29]:
time_horizon = 100
T = 6
#targets = util.gen_targets(T, low=1, high=10)
targets = util.gen_tar_with_len(T)
values = tuple((v, v) for v in targets)
print(values)
#obs = gen_observabilities_correlated_with_values(values)
#obs = {0: 0.75, 1: 0.75, 2: 0.75, 3: 0.75, 4: 0.75}
#obs_vect = [obs.get(i) for i in range(len(targets))]
#generating pure strat and some rand mixed strat stochastic attackers
distributions = []
distributions.append(tuple([1, 0, 0, 0, 0, 0]))
distributions.append(tuple([0, 1, 0, 0, 0, 0]))
distributions.append(tuple([0, 0, 1, 0, 0, 0]))
distributions.append(tuple([0, 0, 0, 1, 0, 0]))
distributions.append(tuple([0, 0, 0, 0, 1, 0]))
distributions.append(tuple([0, 0, 0, 0, 0, 1]))
for i in range(10):
    distributions.append(tuple(util.gen_distr(T)))
print(distributions)

((0.997, 0.997), (0.949, 0.949), (0.807, 0.807), (0.755, 0.755), (0.768, 0.768), (0.808, 0.808))
[(1, 0, 0, 0, 0, 0), (0, 1, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0), (0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 1), (0.05694305694305694, 0.00899100899100899, 0.0989010989010989, 0.12187812187812186, 0.05994005994005993, 0.6533466533466533), (0.4675324675324675, 0.05394605394605394, 0.10289710289710288, 0.27372627372627373, 0.013986013986013984, 0.08791208791208789), (0.1751751751751752, 0.0810810810810811, 0.03703703703703704, 0.11011011011011013, 0.4134134134134134, 0.1831831831831832), (0.126, 0.032, 0.139, 0.082, 0.046, 0.575), (0.11288711288711291, 0.2547452547452548, 0.28771228771228774, 0.15484515484515485, 0.16783216783216787, 0.02197802197802198), (0.26, 0.221, 0.295, 0.154, 0.036, 0.034), (0.131, 0.355, 0.27, 0.111, 0.08, 0.053), (0.0019980019980019984, 0.008991008991008992, 0.38361638361638367, 0.05594405594405595, 0.17682317682317683, 0.37262737262737267), (0.131, 0.014, 0.

In [30]:
g1 = game.Game(values, time_horizon)
g1.attackers = [1]
g1.defenders = [0]
d1 = [bm.FB(g1, 0)]
p1 = [attackers.StackelbergAttacker(g1, 1)]
stoc_prof = []
for d in distributions:
    stoc_prof.append(attackers.StochasticAttacker(g1, 1, 1, *d))
suqr_prof = []
for i in range(10):
    suqr_prof.append(attackers.SUQR(g1, 1))
a1 = [deepcopy(stoc_prof[10])] #deepcopy(p1)
print("Real attacker is: ", util.print_adv(a1[0]))
profiles1 = stoc_prof + suqr_prof + p1
g1.set_players(d1, a1, profiles1)

#go1 = game.GameWithObservabilities(values, time_horizon)
#go1.attackers = [1]
#go1.defenders = [0]
#a1 = [attackers.StackelbergAttacker(go1, 1)]
#d1 = [bm.FB(go1, 0)]
#p1 = [attackers.StackelbergAttacker(go1, 1)]
#stoc_prof1 = []
#for d in distributions:
#    stoc_prof1.append(attackers.StochasticAttacker(go1, 1, 1, *d))
#att_prof1 = [attackers.StackelbergAttacker(go1, 1)]
#stoc_prof_and_att1 = stoc_prof1 + att_prof1
#go1.set_players(d1, a1, stoc_prof_and_att1)

Real attacker is:  sto1-0.11288711288711291-0.2547452547452548-0.28771228771228774-0.15484515484515485-0.16783216783216787-0.02197802197802198


In [31]:
def print_header(targets, profiles):
    return "Name,T," + ",".join(str(i) for i in range(len(targets))) + ",Defender,Attacker," + ",".join(["Profile" for x in range(len(profiles))]) + "\n" 

In [32]:
def print_row(targets, game_type, time_horizon, d, a, p):
    return ",".join([str(i) for i in ([d+"_vs_"+a+"_"+game_type] + [time_horizon] + targets + [d] + [a] + [x for x in p]) ])+"\n"

In [33]:
(print_header(targets, profiles1))

'Name,T,0,1,2,3,4,5,Defender,Attacker,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile\n'

In [34]:
defender =["FB1", "EXP1", "MAB1"]#, "FR1-1"

In [35]:
profile_strings = []
for p in profiles1:
    profile_strings.append(util.print_adv(p))
#profiles1 = []
#for p in stoc_prof_and_att1:
#    profiles1.append(util.print_adv(p))

In [36]:
experiments_folder = "../NewExperiments"
exp_subfolder = "/vsStackelberg"
batch_name = "exp_vs_mab_vs_fb - varying obs"
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Generating the csv and checking everything is fine

In [37]:
with open(batch_path, "w+") as f:
    f.write(print_header(targets, profiles1))
    f.write(print_row(targets, "no_obs", time_horizon,defender[0],util.print_adv(a1[0]),profile_strings))
    f.write(print_row(targets, "no_obs", time_horizon,defender[1],util.print_adv(a1[0]),profile_strings))
    f.write(print_row(targets, "no_obs", time_horizon,defender[2],util.print_adv(a1[0]),profile_strings))
#    f.write(print_row(targets, "no_obs", time_horizon,defender[3],"sta1",profile_strings))
#    f.write(print_row(targets, "obs", time_horizon,defender[1],"sta1",profiles1, obs_vect))

In [12]:
pd.read_csv(batch_path)

,Name,T,0,1,2,3,4,5,Defender,Attacker,...,Profile.23,Profile.24,Profile.25,Profile.26,Obs0,Obs1,Obs2,Obs3,Obs4,Obs5
0,FB1_vs_sta1_obs,100,0.805,0.762,0.942,0.837,0.977,0.893,FB1,sta1,...,suqr1-7.187-0.838,sto1-0-0-0-0-1-0,sta1,sto1-0-1-0-0-0-0,0.9,0.9,0.9,0.9,0.9,0.9
1,EXP1_vs_sta1_obs,100,0.805,0.762,0.942,0.837,0.977,0.893,EXP1,sta1,...,suqr1-7.187-0.838,sto1-0-0-0-0-1-0,sta1,sto1-0-1-0-0-0-0,0.9,0.9,0.9,0.9,0.9,0.9
2,MAB1_vs_sta1_obs,100,0.805,0.762,0.942,0.837,0.977,0.893,MAB1,sta1,...,suqr1-7.187-0.838,sto1-0-0-0-0-1-0,sta1,sto1-0-1-0-0-0-0,0.9,0.9,0.9,0.9,0.9,0.9


### Parsing the csv to generate a batch containing the configurations and running them

In [37]:
#os.mkdir("../NewExperiments/Exp2/belief_max_obs_vs_no_obs")

In [38]:
b = runner.Batch(batch_path, experiments_folder + exp_subfolder)

In [39]:
b.parse_batch()

In [40]:
batches = []
batches.append(b)

In [41]:
with concurrent.futures.ProcessPoolExecutor(None) as executor:
    futures = {}
    logger.info(str(datetime.datetime.now()))
    for b in batches:
        b.run(futures, executor, n=2)

### Collecting results of the runs

In [42]:
with open('progress', 'w') as f:
    with redirect_stdout(f):
        for b in batches:
            b.collect(futures)
            confs = b.configurations
            print(b.results_folder_path)
            dlist = [{"name": c.game.players[0].__class__.name,
                      "avgs": c.stats["exp_regret"], 
                      "lb": c.stats["lb_exp_regret"],
                      "ub": c.stats["ub_exp_regret"]}
                     for c in confs]
            util.plot_dicts(dlist, save=True, show=False, semilog=True,
                            path=b.results_folder_path,
                            title=confs[0].game.players[1].__class__.name)

### Checking everything was fine by running the two configurations "step-by-step"

In [14]:
gm = deepcopy(batches[0].configurations[0].game)

In [15]:
gm

<GameWithObservabilities values:[[0.805, 0.805], [0.762, 0.762], [0.9420000000000001, 0.9420000000000001], [0.8370000000000001, 0.8370000000000001], [0.977, 0.977], [0.893, 0.893]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>} time_horizon:100>

In [16]:
e = runner.Experiment(gm)

In [17]:
for i in range(900):
    e.run_interaction()
    print(e.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e.game.history[-1])
    #print(list(e.agent.belief.pr.items())[2])

TypeError: can't multiply sequence by non-int of type 'float'

In [27]:
e.game

<Game values:[[5.0, 5.0], [3.0, 3.0], [8.0, 8.0], [4.0, 4.0], [9.0, 9.0]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>} time_horizon:1000>

In [28]:
e.agent.belief.loglk

{<StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.6848977664046223,
 <StochasticAttacker id:1 resources:1>: -1.6094379124340925,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.688879454114018,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.688879454114018,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.190303301163861,
 <StochasticAttacker id:1 resources:1>: -1.2740214327993638,
 <StochasticAttacker id:1 resources:1>: -0.09126790561131946,
 <StochasticAttacker id:1 resources:1>: -3.6848977664046223,
 <StochasticAttacker id:1 resources:1>: -0.1133238910766163,
 <StochasticAttacker id:1 resources:1>: -3.32124264636016,
 <StochasticAttacker id:1 resources:1>: -1.6956695963863746,
 <Stochasti

In [29]:
gm1 = deepcopy(batches[0].configurations[1].game)

In [30]:
gm1

<GameWithObservabilities values:[[5.0, 5.0], [3.0, 3.0], [8.0, 8.0], [4.0, 4.0], [9.0, 9.0]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>} time_horizon:1000>

In [31]:
e1 = runner.Experiment(gm1)

In [32]:
for i in range(900):
    e1.run_interaction_with_observations()
    print(e1.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e1.game.history[-1])
    print(e1.game.observation_history[-1])
    #print(list(e.agent.belief.pr.items())[2])

{0: (0, 0, 0, 1, 0), 1: [0, 0, 0, 0, 1]}
{1: [4], 0: [3]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: (0, 0, 0, 0, 1), 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}


{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 0, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 

In [33]:
e1.game

<GameWithObservabilities values:[[5.0, 5.0], [3.0, 3.0], [8.0, 8.0], [4.0, 4.0], [9.0, 9.0]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>} time_horizon:1000>

In [34]:
e1.agent.belief.loglk

{<StochasticAttacker id:1 resources:1>: -1.1129739779822934,
 <StochasticAttacker id:1 resources:1>: -3.688879454114005,
 <StochasticAttacker id:1 resources:1>: -3.31473542646758,
 <StochasticAttacker id:1 resources:1>: -3.676243002079115,
 <StochasticAttacker id:1 resources:1>: -0.10840989264820851,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -1.6094379124340936,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StackelbergAttacker id:1 resources:1>: 0.0,
 <StochasticAttacker id:1 resources:1>: -3.676243002079115,
 <StochasticAttacker id:1 resources:1>: -0.8422254363670488,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -1.0222401724074046,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -1.5353064195098325,
 <StochasticAttacker id:1 resources:1>: -0.13063341972759857,
 <Stochast

In [33]:
pfs = ((0.805,0.805),(0.762,0.76),(0.942,0.942),(0.837,0.837),(0.977,0.977),(0.893,0.893))
obvs = {0:0.5,1:0.5,2:0.5,3:0.5,4:0.5,5:0.5}

In [31]:
def get_player_payoffs(player_index, moves):
        
        covered_targets = set(t for t in moves[0])

        if player_index == 1:
            hit_targets = set(t for t in moves[player_index]
                              if t not in covered_targets)
            return [v[player_index] * (i in hit_targets)
                    for (i, v) in enumerate(pfs)]
        elif player_index == 0:
            all_hit_targets = set(t for t in moves[1]
                                  if t not in covered_targets)
            return [-(v[player_index]) * (i in all_hit_targets)
                    for (i, v) in enumerate(pfs)]

def exp_loss(strategy_vec, **kwargs):
        if isinstance(strategy_vec, dict):
            exp_loss = 0
            for i, s_d in enumerate(strategy_vec[0]):
                for j, s_a in enumerate(strategy_vec[1]):
                    if j != i:
                        guard = 1
                    else:
                        guard = 1 - obvs.get(i)
                    payoffs = get_player_payoffs(0, {0: [i], 1:[j]})
                    k = s_d * s_a * payoffs[j] * guard
                    print(i, j, guard, k)
                    exp_loss -= k
            return exp_loss

In [32]:
str_v = {0:[0.11, 0.06, 0.24, 0.14, 0.26, 0.19], 1:[0, 1, 0, 0, 0, 0]}
exp_loss(str_v)

0 0 0.5 -0.0
0 1 1 -0.08382
0 2 1 -0.0
0 3 1 -0.0
0 4 1 -0.0
0 5 1 -0.0
1 0 1 -0.0
1 1 0.5 -0.0
1 2 1 -0.0
1 3 1 -0.0
1 4 1 -0.0
1 5 1 -0.0
2 0 1 -0.0
2 1 1 -0.18288
2 2 0.5 -0.0
2 3 1 -0.0
2 4 1 -0.0
2 5 1 -0.0
3 0 1 -0.0
3 1 1 -0.10668000000000001
3 2 1 -0.0
3 3 0.5 -0.0
3 4 1 -0.0
3 5 1 -0.0
4 0 1 -0.0
4 1 1 -0.19812000000000002
4 2 1 -0.0
4 3 1 -0.0
4 4 0.5 -0.0
4 5 1 -0.0
5 0 1 -0.0
5 1 1 -0.14478
5 2 1 -0.0
5 3 1 -0.0
5 4 1 -0.0
5 5 0.5 -0.0


0.71628

In [45]:
str_v[0]

[0.11, 0.06, 0.24, 0.14, 0.26, 0.19]

In [46]:
for i, s_d in enumerate(str_v[0]):
    print(i, s_d)

0 0.11
1 0.06
2 0.24
3 0.14
4 0.26
5 0.19


In [58]:
print(k)

[]


In [34]:
pfs[1][1]

0.76

In [11]:
print(batch_path)

../NewExperiments/vsStackelberg/exp_vs_mab_vs_fb.csv
